In [3]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

# 多请求查询
基于距离的矢量数据库检索将查询嵌入(表示)在高维空间中，并根据“距离”找到相似的嵌入文档。但是，由于查询措辞的细微变化，或者如果嵌入没有很好地捕获数据的语义，检索可能会产生不同的结果。提示工程/调优有时是为了手动解决这些问题，但可能很繁琐。
`MultiQueryRetriever`通过使用LLM从不同的角度为给定的用户输入查询生成多个查询，从而使提示调优过程自动化。对于每个查询，它检索一组相关文档，并在所有查询中获取唯一联合，以获得更大的潜在相关文档集。通过在同一个问题上生成多个透视图，`MultiQueryRetriever`可能能够克服基于距离的检索的一些限制，并获得更丰富的结果集。

In [4]:
from langchain_community.document_loaders import TextLoader
# Build a sample vectorDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# # Load blog post
# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# data = loader.load()
loader = TextLoader("../../data/2015北京年报.txt",encoding='utf-8')
documents = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) # overlap失败
splits = text_splitter.split_documents(documents)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [5]:
splits

[Document(page_content='北京住房公积金2015年年度报告\n发布日期：2016-03-31\n根据国务院《住房公积金管理条例》和住房城乡建设部、财政部、人民银行《关于健全住房公积金信息披露制度的通知》(建金〔2015〕26号)的规定，经北京住房公积金管理委员会审议通过，现将北京住房公积金2015年年度报告予以公布。\n\n一、机构概况\n（一) 住房公积金管理委员会\n北京住房公积金管理委员会（以下简称管委会）有32名成员。第十五次管委会全体会议审议通过了2014年住房公积金归集使用计划执行情况和2015年归集使用计划，2014年住房公积金增值收益收支情况和2015年收支计划，2015年度住房公积金年度缴存比例。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='（二）住房公积金管理中心\n北京住房公积金管理中心（以下简称管理中心）为北京市政府直属的全额拨款事业单位，主要负责北京地区住房公积金的归集、管理、使用和会计核算。目前中心设置3个分中心，内设13个处室和工会；垂直管理20个分支机构（18个管理部和住房公积金贷款中心、结算中心）；下设3个直属事业单位：北京住房公积金客户服务中心、北京市住房贷款担保中心、北京市住房贷款个人信用信息服务中心。分支机构营业网点详情请参见北京住房公积金网http://www.bjgjj.gov.cn/。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='二、业务运行情况\n（一）缴存\n职工当月缴存的住房公积金为单位和本人月缴存额之和，金额为职工上一年月平均工资乘以缴存比例。2015住房公积金年度缴存比例为12%。单位和个人缴存上限分别为2014年北京市职工月平均工资的3倍乘以缴存比例，职工和单位月缴存额上限均为2327元。职工住房公积金月缴存基数不得低于北京市人力资源和社会保障局公布的2015年北京市最低工资标准1720元；下岗、内退等类似情况职工月缴存基数不得低于2015年北京市基本生活费1204元。经管委会批准，同意98个单位降低缴存比例和缓缴住房公积金。\n为维护职工合法权益，管理中心与工商、地税、人力社保

## 简单的使用
指定要用于生成查询的LLM，检索器将完成其余的工作。

In [6]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "2015年提取业务的情况"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [7]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [8]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 2015年业务提取的情况如何？', '2. 2015年的业务提取情况有哪些？', '3. 关于2015年的业务提取情况，你有什么了解？']


4

In [9]:
unique_docs

[Document(page_content='（二）提取业务\n2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='（三）历史遗留风险资产\n截至2015年底，不存在历史遗留风险资产。\n\n\n五、社会经济效益\n（一）缴存业务\n2015年住房公积金实缴人数和缴存额增长率分别为14.0%和13.8%。缴存职工的构成情况：按单位性质，国家机关和事业单位占18.7%，国有企业占20.2%，城镇集体企业占0.7%，外商投资企业占11.8%，城镇私营企业及其他城镇企业占36.4%，民办非企业单位和社会团体占0.7%，其他占11.5%。\n按收入水平，低收入群体（住房公积金缴存基数低于2014年本市职工平均工资）占53.1%，中等收入群体（住房公积金缴存基数介于2014年本市职工平均工资1倍-3倍之间）占38.5 %，高收入群体（住房公积金缴存基数高于2014年本市职工平均工资3倍）占8.4%。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='（四）购买国债\n2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。\n\n（五）资金存储\n截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。\n\n（六）资金运用率\n2015年住房公积金提取、贷款业务量大幅增加，结余资金大幅减少，资金使用效率进一步提升。截至2015年底，资金运用率（住房公积金个人住房贷款余额、项目贷款余额和购买国债余额的总和与缴存余额的比率）89.6%，比上年同期增加9.9

## 自定义提示词
还可以提供提示词和输出解析器，将结果拆分为查询列表。

In [10]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries 输出解析器将把LLM结果拆分为查询列表
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""你是一名中文AI助手。你的任务是根据用户给定的问题生成3个不同版本从向量数据库中检索相关文档的新问题。
    需要对用户的问题进行多视角分析，你的目标是帮助用户解决相似度搜索的限制。最后的答案用中文回答,将生成的问题用换行符分隔开。
    用户给定的最初问题{question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "2015年提取业务的情况?"

In [11]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query="2015年的提取总额?"
)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 2015年的提取总额是多少？', '2. 请问2015年的提取总额是多少？', '3. 你知道2015年的提取总额是多少吗？']


4

In [12]:
unique_docs

[Document(page_content='（二）提取业务\n2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='（二）提取\n2015年缴存人提取住房公积金920.62亿元，其中购房提取792.67亿元，租房提取15.87亿元，离休和退休提取90.49亿元，其他提取21.59亿元。提取占当年缴存额的比率为71%，同比增加3.6个百分点。截至2015年底，提取总额5002.76亿元，同比增长22.6%。\n为严厉打击违规提取、骗提住房公积金的行为，维护北京住房公积金管理的良好秩序，经调查核实，管理中心对存在造假违规骗提住房公积金行为的26家单位进行了查处；同时会同有关执法部门对违法行为涉及的12个电话号码依法进行了停机处理，并向社会公布，有效遏制了违法违规行为。', metadata={'source': '../../data/2015北京年报.txt'}),
 Document(page_content='（四）购买国债\n2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。\n\n（五）资金存储\n截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。\n\n（六）资金运用率\n2015年住房公积金提取、贷款业务量大幅增加，结余资金大幅减少，资金使用效率进一步提升。截至2015年底，资金运用率（住房公积金个人住房贷款余额、项目贷款余额和购买国债余额的总和与缴存余额的比率）89.6%，比上年同期增加9.9个百分点。\n\n\n三、主要财务数据\n（一）业务收入\n2015全年,住房公积金业务收入共计1088734.64万元，同比增